In [1]:
import os
import openai

from decouple import Config, RepositoryEnv
config = Config(RepositoryEnv("/workspaces/codespaces-jupyter/.env"))

In [2]:
openai.api_key = config("OPENAI_API_KEY")

In [3]:
response = openai.Completion.create(
    model="text-davinci-003",
    prompt="What are the oceans adjacent to Asia?",
    max_tokens=300
    )

In [8]:
print(response['choices'][0]['text'])



The oceans adjacent to Asia are the Arctic Ocean, Indian Ocean, Pacific Ocean, and South China Sea.


In [9]:
import pandas as pd

df = pd.read_csv('./01-NLP-to-SQL/data/sales_data_sample.csv')

In [10]:
df

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,20,100.00,2244.40,12/2/2004 0:00,4,12,2004,Ships,(91) 555 94 44,"C/ Moralzarzal, 86",Madrid,NaN,28034,Spain,Freyre,Diego
2819,10373,29,100.00,3978.51,1/31/2005 0:00,1,1,2005,Ships,981-443655,Torikatu 38,Oulu,NaN,90110,Finland,Koskitalo,Pirkko
2820,10386,43,100.00,5417.57,3/1/2005 0:00,1,3,2005,Ships,(91) 555 94 44,"C/ Moralzarzal, 86",Madrid,NaN,28034,Spain,Freyre,Diego
2821,10397,34,62.24,2116.16,3/28/2005 0:00,1,3,2005,Ships,61.77.6555,1 rue Alsace-Lorraine,Toulouse,NaN,31000,France,Roulet,Annette


In [12]:
import sqlalchemy
from sqlalchemy import create_engine, text

In [13]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [14]:
data = df.to_sql(name="Sales", con=temp_db)

2023-05-07 09:31:26,301 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 09:31:26,302 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-05-07 09:31:26,303 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 09:31:26,305 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-05-07 09:31:26,305 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 09:31:26,307 INFO sqlalchemy.engine.Engine ROLLBACK
2023-05-07 09:31:26,310 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 09:31:26,312 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-05-07 09:31:26,312 INFO sqlalche

In [16]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT * FROM Sales;"))

2023-05-07 09:33:02,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 09:33:02,630 INFO sqlalchemy.engine.Engine SELECT * FROM Sales;
2023-05-07 09:33:02,630 INFO sqlalchemy.engine.Engine [generated in 0.00239s] ()
2023-05-07 09:33:02,632 INFO sqlalchemy.engine.Engine ROLLBACK


In [17]:
result.fetchone()

(0, 10107, 30, 95.7, 2871.0, '2/24/2003 0:00', 1, 2, 2003, 'Motorcycles', '2125557818', '897 Long Airport Avenue', 'NYC', 'NY', '10022', 'USA', 'Yu', 'Kwai')

In [18]:
def create_table_definition(df):
    prompt = """### sqlite SQL table, with its properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))

    return prompt

In [19]:
print(create_table_definition(df))

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    


In [20]:
def prompt_input():
    nlp_text = input("Enter the info you want: ")
    return nlp_text

In [21]:
prompt_input()

'get all sales per quarter'

In [22]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_str = f"### A query to answer: {query_prompt}\nSELECT"
    
    return definition + query_init_str

In [39]:
nlp_text = prompt_input()
combine_prompts(df, nlp_text)

'### sqlite SQL table, with its properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)\n    #\n    ### A query to answer: total sales per QTR_ID\nSELECT'

In [40]:
print(combine_prompts(df, nlp_text))

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    ### A query to answer: total sales per QTR_ID
SELECT


In [42]:
response = openai.Completion.create(
    model='davinci',
    prompt=combine_prompts(df, nlp_text),
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0,
    stop=['#', ';']
)

In [43]:
response['choices'][0]['text']

' SUM(SALES) AS SALES_QTR_ID\nFROM Sales\nGROUP BY QTR_ID\nORDER BY QTR_ID'

In [44]:
def handle_response(response):
    query_body = response['choices'][0]['text']
    if query_body.startswith(" "):
        query_body = "SELECT" + query_body
    return query_body

In [45]:
handle_response(response)

'SELECT SUM(SALES) AS SALES_QTR_ID\nFROM Sales\nGROUP BY QTR_ID\nORDER BY QTR_ID'

In [46]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-05-07 10:55:30,386 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 10:55:30,388 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) AS SALES_QTR_ID
FROM Sales
GROUP BY QTR_ID
ORDER BY QTR_ID
2023-05-07 10:55:30,389 INFO sqlalchemy.engine.Engine [generated in 0.00235s] ()
2023-05-07 10:55:30,391 INFO sqlalchemy.engine.Engine ROLLBACK


In [47]:
result.all()

[(2350817.7300000004,),
 (2048120.2999999986,),
 (1758910.8099999994,),
 (3874780.01,)]